# 환경설정

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [5]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "./chromedriver"
# driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

# 음식점 별 상세보기를 리스트로 가져오기

In [6]:
import re
import time

# brew 로 설치된 chromedriver의 path (Mac)
path = '/usr/local/bin/chromedriver'

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
excutable_path = 'chromedriver.exe'

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://map.kakao.com/"

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
# driver = webdriver.Chrome(path)
driver = webdriver.Chrome(executable_path=excutable_path)

# 카카오 지도에 접속합니다
driver.get(source_url)

# 검색창에 검색어를 입력합니다
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
searchbox.send_keys("인사동 맛집")

# 검색버튼을 눌러서 결과를 가져옵니다
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
driver.execute_script("arguments[0].click();", searchbutton)

# 검색 결과를 가져올 시간을 기다립니다
time.sleep(2)

# 검색 결과의 페이지 소스를 가져옵니다
html = driver.page_source


driver.find_element(By.XPATH, '/html/body/div[10]/div/div/div/span').click()
time.sleep(2)
#더보기 클릭 
driver.find_element(By.XPATH, "//*[@id='info.search.place.more']").click()
time.sleep(2)
# 1~ 5페이지 링크 얻기
page_urls = []
for i in range(1,6):
    page = driver.find_element(By.ID, 'info.search.page.no' + str(i))
    page.click()
    time.sleep(2)
    urls = driver.find_elements(By.LINK_TEXT, "상세보기")
    for j in range(len(urls)):
        url = urls[j].get_attribute('href')
        print(url)
        page_urls.append(url)
        


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19828\3317620806.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19828\3317620806.py:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19828\3317620806.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")


KeyboardInterrupt: 

## 상세보기 링크 개수 확인

In [26]:
len(page_urls)

75

## 링크를 하나씩 열어 음식점별 정보를 DF형태로 저장

In [24]:
columns = ['name','addresse','cat1','cat2','review_num','score', 'review']
df = pd.DataFrame(columns=columns)

# driver = webdriver.Chrome(path)  # for Mac
driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
for page_url in page_urls:

    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    time.sleep(5)
    #사업장명
    name =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
    # 사업장 주소
    address = driver.find_element(By.CLASS_NAME, "txt_address").text
    # 음식 종류
    cat1 =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]').text
    # 메뉴
    cat2 = []
    menus = driver.find_elements(By.CLASS_NAME, "info_menu")
    for menu in menus:
        cat2.append(menu.text)
    if driver.find_elements(By.CLASS_NAME, "box_grade_off"): #후기를 제공하지 않는 맛집 넘기기
        continue
    else:
        #리뷰수
        review_num = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[2]/span').text
        # 첫 페이지 리뷰를 크롤링합니다
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

        # 별점을 가져옵니다.
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

        # 리뷰를 가져옵니다.
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

        for rate, review in zip(rates, reviews):
            row = [name,address,cat1,cat2,review_num,rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=df.columns)
            df = df.append(series, ignore_index=True)


        # 2-5페이지의 리뷰를 크롤링합니다
        for button_num in range(2, 6):
            # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
            try:
                another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
                another_reviews.click()
                time.sleep(2)

                # 페이지 리뷰를 크롤링합니다
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

                # 별점을 가져옵니다.
                rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
                # 리뷰를 가져옵니다.
                reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

                for rate, review in zip(rates, reviews):
                    row = [name,address,cat1,cat2,review_num,rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=df.columns)
                    df = df.append(series, ignore_index=True)
            except:
                break    
driver.close()

C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1604348524.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1604348524.py:48: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")


# 위 과정을 한번에 처리하고 여러 키워드를 반복해서 돌리기

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

import re
import time

# brew 로 설치된 chromedriver의 path (Mac)
path = '/usr/local/bin/chromedriver'

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
excutable_path = 'chromedriver.exe'

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://map.kakao.com/"

# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
# driver = webdriver.Chrome(path)

# 카카오 지도에 접속합니다

regions = ['충정로','시청','동대문','신당','왕십리','건대입구','성수','강변'] #불러올 키워드
for region in regions:
    driver = webdriver.Chrome(executable_path=excutable_path)
    driver.get(source_url) # 받아온 링크를 차례로 열기
    
    # 검색창에 검색어를 입력합니다
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
    searchbox.send_keys("{0} 맛집".format(region))

    # 검색버튼을 눌러서 결과를 가져옵니다
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
    driver.execute_script("arguments[0].click();", searchbutton)

    # 검색 결과를 가져올 시간을 기다립니다
    time.sleep(2)

    # 검색 결과의 페이지 소스를 가져옵니다
    html = driver.page_source


    driver.find_element(By.XPATH, '/html/body/div[10]/div/div/div/span').click() #소개 팝업 제거
    time.sleep(2)
    
    #더보기 클릭 (클릭하지 않으면 첫페이지인 15개만 나온다)(2페이지로 이동하기와 같다)
    driver.find_element(By.XPATH, "//*[@id='info.search.place.more']").click()
    time.sleep(2)
    
    # 1~ 5페이지 링크 얻기
    page_urls = []
    for i in range(1,6):
        page = driver.find_element(By.ID, 'info.search.page.no' + str(i))
        page.click()
        time.sleep(2)
        urls = driver.find_elements(By.LINK_TEXT, "상세보기")
        for j in range(len(urls)):
            url = urls[j].get_attribute('href')
            print(url)
            page_urls.append(url)

    columns = ['name','addresse','cat1','cat2','review_num','score', 'review']
    #사업장명, 주소, 음식종류1,음식종류2(메뉴),리뷰수,별점,리뷰
    df = pd.DataFrame(columns=columns)

    # driver = webdriver.Chrome(path)  # for Mac
    driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
    for page_url in page_urls:

        # 상세보기 페이지에 접속합니다
        driver.get(page_url)
        time.sleep(5)
        #사업장명
        name =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
        # 사업장 주소
        address = driver.find_element(By.CLASS_NAME, "txt_address").text
        # 음식 종류
        cat1 =  driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/span[1]').text
        # 메뉴
        cat2 = []
        menus = driver.find_elements(By.CLASS_NAME, "info_menu")
        for menu in menus:
            cat2.append(menu.text)
        if driver.find_elements(By.CLASS_NAME, "box_grade_off"): #후기를 제공하지 않는 맛집 넘기기
            continue
        else:
            #리뷰수
            review_num = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[2]/span').text
            # 첫 페이지 리뷰를 크롤링합니다
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

            # 별점을 가져옵니다.
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

            # 리뷰를 가져옵니다.
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
            
            #데이터프레임으로 정리합니다.
            for rate, review in zip(rates, reviews):
                row = [name,address,cat1,cat2,review_num,rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)


            # 2-5페이지의 리뷰를 크롤링합니다 과정은 위와 같습니다.
            for button_num in range(2, 6):
                # 오류가 나는 경우(리뷰 페이지가 없는 경우), 수행하지 않습니다.
                try:
                    another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
                    another_reviews.click()
                    time.sleep(2)

                    # 페이지 리뷰를 크롤링합니다
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

                    # 별점을 가져옵니다.
                    rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
                    # 리뷰를 가져옵니다.
                    reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

                    for rate, review in zip(rates, reviews):
                        row = [name,address,cat1,cat2,review_num,rate.text[0], review.find(name="span").text]
                        series = pd.Series(row, index=df.columns)
                        df = df.append(series, ignore_index=True)
                except:
                    break    
    driver.close()
    df.to_csv('{0}.csv'.format(region), index=False) #index label을 따로 저장하지 않기

C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1369155949.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)
C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1369155949.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1369155949.py:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")


https://place.map.kakao.com/25880887
https://place.map.kakao.com/582437543
https://place.map.kakao.com/17434509
https://place.map.kakao.com/27487568
https://place.map.kakao.com/16418514
https://place.map.kakao.com/1015060956
https://place.map.kakao.com/8583486
https://place.map.kakao.com/12441941
https://place.map.kakao.com/27049627
https://place.map.kakao.com/9991132
https://place.map.kakao.com/459071226
https://place.map.kakao.com/26482230
https://place.map.kakao.com/1662439887
https://place.map.kakao.com/1019082982
https://place.map.kakao.com/398233724
https://place.map.kakao.com/1922765079
https://place.map.kakao.com/21235462
https://place.map.kakao.com/26452497
https://place.map.kakao.com/10788220
https://place.map.kakao.com/2126219174
https://place.map.kakao.com/632167413
https://place.map.kakao.com/26081926
https://place.map.kakao.com/69144095
https://place.map.kakao.com/612925043
https://place.map.kakao.com/16042503
https://place.map.kakao.com/1280840657
https://place.map.kakao

C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1369155949.py:61: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
C:\Users\max_s\AppData\Local\Temp/ipykernel_22828/1369155949.py:104: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")


https://place.map.kakao.com/1545526052
https://place.map.kakao.com/1102847581
https://place.map.kakao.com/8178288
https://place.map.kakao.com/1920359216
https://place.map.kakao.com/620041434
https://place.map.kakao.com/871390985
https://place.map.kakao.com/220828644
https://place.map.kakao.com/26470073
https://place.map.kakao.com/1239408575
https://place.map.kakao.com/815706578
https://place.map.kakao.com/503488711
https://place.map.kakao.com/2144279590
https://place.map.kakao.com/21397192
https://place.map.kakao.com/21324404
https://place.map.kakao.com/252889396
https://place.map.kakao.com/26966188
https://place.map.kakao.com/18234144
https://place.map.kakao.com/1111308175
https://place.map.kakao.com/18474517
https://place.map.kakao.com/115914510
https://place.map.kakao.com/738920496
https://place.map.kakao.com/11902329
https://place.map.kakao.com/21604330
https://place.map.kakao.com/1745256384
https://place.map.kakao.com/10305201
https://place.map.kakao.com/13491657
https://place.map

https://place.map.kakao.com/12265333
https://place.map.kakao.com/142477401
https://place.map.kakao.com/937765562
https://place.map.kakao.com/24019212
https://place.map.kakao.com/16038487
https://place.map.kakao.com/26458155
https://place.map.kakao.com/1454921402
https://place.map.kakao.com/588104249
https://place.map.kakao.com/1285252800
https://place.map.kakao.com/9112049
https://place.map.kakao.com/422862484
https://place.map.kakao.com/21358629
https://place.map.kakao.com/761704459
https://place.map.kakao.com/1208917358
https://place.map.kakao.com/1465449376
https://place.map.kakao.com/16874815
https://place.map.kakao.com/27142658
https://place.map.kakao.com/2096559251
https://place.map.kakao.com/8211598
https://place.map.kakao.com/26615339
https://place.map.kakao.com/16617878
https://place.map.kakao.com/18474292
https://place.map.kakao.com/567965853
https://place.map.kakao.com/1129598832
https://place.map.kakao.com/1281524879
https://place.map.kakao.com/27440269
https://place.map.ka

https://place.map.kakao.com/9072805
https://place.map.kakao.com/22873800
https://place.map.kakao.com/1907427784
https://place.map.kakao.com/2015369599
https://place.map.kakao.com/21360263
https://place.map.kakao.com/1448221336
https://place.map.kakao.com/27557761
https://place.map.kakao.com/437017954
https://place.map.kakao.com/24765606
https://place.map.kakao.com/10455440
https://place.map.kakao.com/899891312
https://place.map.kakao.com/196485088
https://place.map.kakao.com/156027137
https://place.map.kakao.com/23384358
https://place.map.kakao.com/1940553585
https://place.map.kakao.com/11841499
https://place.map.kakao.com/7821643
https://place.map.kakao.com/27390816
https://place.map.kakao.com/7831256
https://place.map.kakao.com/3155649
https://place.map.kakao.com/1091101711
https://place.map.kakao.com/20883333
https://place.map.kakao.com/1024200053
https://place.map.kakao.com/8128653
https://place.map.kakao.com/879311630
https://place.map.kakao.com/15524383
https://place.map.kakao.co

## 데이터 확인하기

In [33]:
df

,name,addresse,cat1,cat2,review_num,score,review
0,황금콩밭,서울 마포구 마포대로16길 9 (우)04205,두부전문점,"[두부짜글이\n8,000, 돼지짜글이\n9,000, 시래기비지탕\n9,000, 두부...",130,5,제발 냄비두부조림 두부구이
1,황금콩밭,서울 마포구 마포대로16길 9 (우)04205,두부전문점,"[두부짜글이\n8,000, 돼지짜글이\n9,000, 시래기비지탕\n9,000, 두부...",130,4,나무랄 데 없는 맛. 근처에서 술 먹을 있으면 들를 것 같습니다.
2,황금콩밭,서울 마포구 마포대로16길 9 (우)04205,두부전문점,"[두부짜글이\n8,000, 돼지짜글이\n9,000, 시래기비지탕\n9,000, 두부...",130,5,맛만보고 별점 드립니다.
3,황금콩밭,서울 마포구 마포대로16길 9 (우)04205,두부전문점,"[두부짜글이\n8,000, 돼지짜글이\n9,000, 시래기비지탕\n9,000, 두부...",130,4,유명해서 기대를 많이했는데 대한민국 손꼽히는 대박맛집 이런 느낌은 아니다 하지만 맛...
4,황금콩밭,서울 마포구 마포대로16길 9 (우)04205,두부전문점,"[두부짜글이\n8,000, 돼지짜글이\n9,000, 시래기비지탕\n9,000, 두부...",130,5,
...,...,...,...,...,...,...,...
935,동대문엽기떡볶이 아현점,서울 마포구 굴레방로 11 1층 (우)04117,동대문엽기떡볶이,"[A SET\n17,000\n엽떡 OR 엽오 OR 반반 중 택1 + 모듬튀김 + 계...",1,1,싸가지 애징간히 없던데ㅋㅋ 포장하러가서 카운터에 아무도 없고 뒤로 보이는 주방에 사...
936,동대문엽기떡볶이 아현점,서울 마포구 굴레방로 11 1층 (우)04117,동대문엽기떡볶이,"[A SET\n17,000\n엽떡 OR 엽오 OR 반반 중 택1 + 모듬튀김 + 계...",1,1,흠 왤까요 포장하러 갔는데... 이 매장에서 배달 시키지 않고 포장 하기 잘했단 생...
937,동대문엽기떡볶이 아현점,서울 마포구 굴레방로 11 1층 (우)04117,동대문엽기떡볶이,"[A SET\n17,000\n엽떡 OR 엽오 OR 반반 중 택1 + 모듬튀김 + 계...",1,1,직원들 마스크도 안 쓰고 서비스 꽝이네요 ㅋㅋㅋ 테이블에 본인들 짐 올려놓고 뭐하자...
938,동대문엽기떡볶이 아현점,서울 마포구 굴레방로 11 1층 (우)04117,동대문엽기떡볶이,"[A SET\n17,000\n엽떡 OR 엽오 OR 반반 중 택1 + 모듬튀김 + 계...",1,1,매장에서 곰팡이 냄새나요... 포장주문하러 가보니 주문하기 싫어집니다..
